In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import pairwise
import pyautogui

In [2]:
background = None

accumulated_weight = 0.5

roi_top    = 0
roi_bottom = 480
roi_right  = 200
roi_left   = 640

hand_hist = None
traverse_point = []
total_rectangle = 9
hand_rect_one_x = None
hand_rect_one_y = None

hand_rect_two_x = None
hand_rect_two_y = None
mouseXmax       = 2725
mouseYmax       = 765

In [3]:
def calc_accum_avg(frame,accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background,accumulated_weight)

In [4]:
def segment(frame,threshold_min=25):
    
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    ret,thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    image,contours,hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    else:
        # ASSUMING THE LARGEST EXTERNAL CONTOUR IN ROI, IS THE HAND
        hand_segment = max(contours,key=cv2.contourArea)
        
        return (thresholded,hand_segment)

In [5]:
def contours(hist_mask_image):
    gray_hist_mask_image = cv2.cvtColor(hist_mask_image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_hist_mask_image, 0, 255, 0)
    _, cont, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return cont

In [6]:
def draw_rect(frame):
    rows, cols, _ = frame.shape
    global total_rectangle, hand_rect_one_x, hand_rect_one_y, hand_rect_two_x, hand_rect_two_y

    hand_rect_one_x = np.array(
        [6 * rows / 20, 6 * rows / 20, 6 * rows / 20, 9 * rows / 20, 9 * rows / 20, 9 * rows / 20, 12 * rows / 20,
         12 * rows / 20, 12 * rows / 20], dtype=np.uint32)

    hand_rect_one_y = np.array(
        [9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20,
         10 * cols / 20, 11 * cols / 20], dtype=np.uint32)

    hand_rect_two_x = hand_rect_one_x + 10
    hand_rect_two_y = hand_rect_one_y + 10

    for i in range(total_rectangle):
        cv2.rectangle(frame, (hand_rect_one_y[i], hand_rect_one_x[i]),
                      (hand_rect_two_y[i], hand_rect_two_x[i]),
                      (0, 255, 0), 1)

    return frame

In [7]:
def hand_histogram(frame):
    global hand_rect_one_x, hand_rect_one_y

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    roi = np.zeros([90, 10, 3], dtype=hsv_frame.dtype)

    for i in range(total_rectangle):
        roi[i * 10: i * 10 + 10, 0: 10] = hsv_frame[hand_rect_one_x[i]:hand_rect_one_x[i] + 10,
                                          hand_rect_one_y[i]:hand_rect_one_y[i] + 10]

    hand_hist = cv2.calcHist([roi], [0, 1], None, [180, 256], [0, 180, 0, 256])
    return cv2.normalize(hand_hist, hand_hist, 0, 255, cv2.NORM_MINMAX)

In [8]:
def hist_masking(frame, hist):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)

    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))
    cv2.filter2D(dst, -1, disc, dst)

    ret, thresh = cv2.threshold(dst, 150, 255, cv2.THRESH_BINARY)

    thresh = cv2.merge((thresh, thresh, thresh))

    return cv2.bitwise_and(frame, thresh)

In [9]:
def centroid(max_contour):
    moment = cv2.moments(max_contour)
    if moment['m00'] != 0:
        cx = int(moment['m10'] / moment['m00'])
        cy = int(moment['m01'] / moment['m00'])
        return cx, cy
    else:
        return None

In [10]:
def farthest_point(defects, contour, centroid):
    if defects is not None and centroid is not None:
        s = defects[:, 0][:, 0]
        cx, cy = centroid

        x = np.array(contour[s][:, 0][:, 0], dtype=np.float)
        y = np.array(contour[s][:, 0][:, 1], dtype=np.float)

        xp = cv2.pow(cv2.subtract(x, cx), 2)
        yp = cv2.pow(cv2.subtract(y, cy), 2)
        dist = cv2.sqrt(cv2.add(xp, yp))

        dist_max_i = np.argmax(dist)

        if dist_max_i < len(s):
            farthest_defect = s[dist_max_i]
            farthest_point = tuple(contour[farthest_defect][0])
            return farthest_point
        else:
            return None

In [11]:
def manage_image_opr(frame, hand_hist):
    hist_mask_image = hist_masking(frame, hand_hist)

    hist_mask_image = cv2.erode(hist_mask_image, None, iterations=2)
    hist_mask_image = cv2.dilate(hist_mask_image, None, iterations=2)

    contour_list = contours(hist_mask_image)
    max_cont = max(contour_list, key=cv2.contourArea)

    cnt_centroid = centroid(max_cont)
    cv2.circle(frame, cnt_centroid, 5, [0, 0, 255], -1)

    if max_cont is not None:
        hull = cv2.convexHull(max_cont, returnPoints=False)
        defects = cv2.convexityDefects(max_cont, hull)
        far_point = farthest_point(defects, max_cont, cnt_centroid)
        cv2.line(frame, cnt_centroid, far_point, [0, 0, 255], 4)
        dist = (cnt_centroid[0] - far_point[0]) ** 2 + (cnt_centroid[1] - far_point[1]) ** 2
        cv2.circle(frame, cnt_centroid,  int(dist ** 0.5), (0, 0, 255), 2)
        if len(traverse_point) < 20:
            traverse_point.append(far_point)
        else:
            traverse_point.pop(0)
            traverse_point.append(far_point)

        return cnt_centroid, far_point, int(dist ** 0.5) + 5

In [12]:
def canSaveOnData(event,x,y,flags,param):
    global canSave, numbers
    if event == cv2.EVENT_LBUTTONDOWN:
        canSave = True
        numbers = 250

In [13]:
from keras.models import load_model
model = load_model("hand40epochs.h5")

Using TensorFlow backend.


In [94]:
is_hand_hist_created = False
num_frames = 0
labels  = ["FiveFingers", "FourFingers", "ThreeFingers", "TwoFingers", "OneFinger", "Closed", "Rock", "Nice", "NanoNano", "HangLoose"]
center, final  = (0, 0), (0, 0)
canAudio       = True
canChangeView  = True
canChangeArea  = True
newView        = True
canMoveView    = True
canMoveOneView = True
canZoom        = True
pastCenter     = (0, 0)

In [95]:
cv2.namedWindow('Tracking')
cv2.setMouseCallback('Tracking', canSaveOnData) 
cam = cv2.VideoCapture(0)
while True:
    ret, frame = cam.read()    
    frame_copy = frame.copy()
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy,'WAIT. GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
    else:
        
        hand = segment(gray)
        
        if hand is not None:
            thresholded , hand_segment = hand
            if k == ord('z'):
                is_hand_hist_created = True
                hand_hist = hand_histogram(roi)
            if is_hand_hist_created:
                try:
                    center, final, radius = manage_image_opr(roi, hand_hist)
                    p1 = (center[0] - radius, center[1] + radius)
                    p2 = (center[0] + radius, center[1] - radius)
                    thresholded = thresholded[p2[1]: p1[1], p1[0]: p2[0]]
                    
                    img = thresholded / 255
                    img = cv2.resize(img, (100, 100))
                    img = np.array([img])
                    img = img.reshape(1, 100, 100, 1)
                    
                    prediction = model.predict_classes(img)
                    if(labels[prediction[0]] == "FiveFingers"):
                        pyautogui.moveTo(mouseXmax - 2 * center[0] * mouseXmax / roi_left, -400 + 2 * center[1] *  mouseYmax / roi_bottom)
                        if (not canClick):
                            canClick = True
                        if (not canAudio):
                            canAudio = True
                        if (not newView):
                            newView  = True
                    elif(labels[prediction[0]] == "Closed" and canClick):
                        canClick = False
                        pyautogui.click(pyautogui.position())
                        if (not canChangeView):
                            pyautogui.keyUp('ctrl')
                            canChangeView = True
                        if (not canMoveView):
                            canMoveView = True
                        if (not canMoveOneView):
                            canMoveOneView = True
                    elif(labels[prediction[0]] == "HangLoose"):
                        if (canAudio):
                            pyautogui.hotkey('ctrl', 'f9')
                            canAudio = False
                    elif(labels[prediction[0]] == "Rock" and pastCenter != (0, 0)):
                        if (center[0] - pastCenter[0] > 4 ):
                            pyautogui.hotkey('ctrl', 'f10')
                        elif (-center[0] + pastCenter[0] > 4):
                            pyautogui.hotkey('ctrl', 'f11')
                    elif (labels[prediction[0]] == "NanoNano" and canChangeView):
                        pyautogui.keyDown('ctrl')
                        pyautogui.press('tab')
                        canChangeView = False
                    elif (labels[prediction[0]] == "ThreeFingers" and newView):
                        pyautogui.hotkey('ctrl', 't')
                        newView = False
                    elif (labels[prediction[0]] == "TwoFingers" and canMoveView):
                        pyautogui.hotkey('alt', 'f7')
                        canMoveView = False
                    elif (labels[prediction[0]] == "Nice" and canMoveOneView):
                        pyautogui.hotkey('alt', 'f')
                        canMoveOneView = False
                    elif (labels[prediction[0]] == "OneFinger" and pastCenter != (0, 0)):
                        if (center[0] - pastCenter[0] > 4 ):
                            pyautogui.hotkey('ctrl', '-')
                        elif (-center[0] + pastCenter[0] > 4):
                            pyautogui.hotkey('ctrl', '+')
                    pastCenter = center   
                    cv2.putText(roi, labels[prediction[0]], (70, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 4)
                except:
                    cv2.putText(roi, "Error", (70, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 4)
            else:
                roi = draw_rect(roi)
            try:
                cv2.imshow('Thresholded',thresholded)
                cv2.imshow("Tracking", roi)
            except:
                center =  center

            
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    num_frames += 1
    
    cv2.imshow('Finger Count',frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()